# Distance and duration request using ORS

This notebook is an example for using different Openrouteservice tools. To get ready all needed packages were imported.

In [1]:
import folium
from folium import FeatureGroup, LayerControl, Map, Marker

from shapely import wkt, geometry
from shapely.geometry import shape

from pprint import pprint

import json

from openrouteservice import client, places, geocoding
from openrouteservice.directions import directions
from openrouteservice.isochrones import isochrones
from openrouteservice.places import places

We have just moved to San Francisco and are looking for the perfect location to get a new home. In a 15 minute walking radius there should be a kindergarten, a supertmarket and a post office. 

# client and isochrones

There are three different suggested locations for our new home. Let's visualize them and the 15 minute walking radius on a map:

In [2]:
api_key = '58d904a497c67e00015b45fc2a6b6872037d44119582ef40cdf264c4'
clnt = client.Client(key=api_key) 

map1 = folium.Map(tiles='Stamen Toner', location=([37.738684, -122.450523]), zoom_start=12)
location_coords = {'first': [-122.430954, 37.792965], 'second': [-122.501636, 37.748653], 'third': [-122.446629, 37.736928]}

# Request of isochrones with 15 minute footwalk.
polygon_borders = []
for border_coords in location_coords:
    isochrone_request = isochrones(client=clnt, locations=location_coords[border_coords], profile='foot-walking', intervals=[900], segments=900)
    folium.GeoJson(isochrone_request).add_to(map1)
    polygon_borders.append(isochrone_request)
for house_coords in location_coords:
    location_coords[house_coords][0], location_coords[house_coords][1] = location_coords[house_coords][1], location_coords[house_coords][0]
    folium.Marker(location_coords[house_coords], icon=folium.Icon(color='lightgray',icon_color='#cc0000', icon='home', prefix='fa'), popup='Location suggestion').add_to(map1)
        
map1

# places

Now let's have a look how many kindergarten, supermarket and post offices there are in each walking radius. 

In [3]:
all_kindergarten = []
all_supermarket = []
all_post = []
for request in polygon_borders:
    all_kindergarten += [clnt.places(request='pois', geojson=request['features'][0]['geometry'], filter_category_ids=[153], sortby='distance')]
    all_supermarket += [clnt.places(request='pois', geojson=request['features'][0]['geometry'], filter_category_ids=[518], sortby='distance')]
    all_post += [clnt.places(request='pois', geojson=request['features'][0]['geometry'], filter_category_ids=[370], sortby='distance')]
    
print("Amount of kindergarten in the first, second and third suggested area: ", [len(kindergarten_amount['features']) for kindergarten_amount in all_kindergarten])
print("Amount of supermarket in the first, second and third suggested area: ", [len(supermarket_amount['features']) for supermarket_amount in all_supermarket])
print("Amount of post offices in the first, second and third suggested area: ", [len(post_amount['features']) for post_amount in all_post])

Amount of kindergarten in the first, second and third suggested area:  [1, 3, 1]
Amount of supermarket in the first, second and third suggested area:  [8, 1, 3]
Amount of post offices in the first, second and third suggested area:  [7, 0, 0]


Okay the first location looks pretty good: There is one kindergarten, eight supermarket and seven post offices. It is the perfect location for our new home!

# geocoding

To get representable names the geocoding api is a useful solution. 

In [4]:
# still the Mollie Stone's problem
name_markets = [(feat["properties"]['osm_tags']['name']) for feat in all_supermarket[0]['features']] 
name_markets[3] = 'Mollie Stone'
x = 0
for feat in all_supermarket[0]['features']:
    feat["properties"]['osm_tags']['name'] = name_markets[x]
    x = x+1
    
name_markets3 = [(feat["properties"]['osm_tags']['name']) for feat in all_supermarket[2]['features']] 
name_markets3[1] = 'Mollie Stone'
x = 0
for feat in all_supermarket[2]['features']:
    feat["properties"]['osm_tags']['name'] = name_markets3[x]
    x = x+1

In [8]:
all_places = {'all_kindergarten': all_kindergarten, 'all_supermarket': all_supermarket, 'all_post': all_post}

#kindergarten_group = FeatureGroup(name='Kindergarten') 
#supermarket_group = FeatureGroup(name='Supermarket')
#post_group = FeatureGroup(name='Post Office')

for all_features in all_places: # all locations and all places
    for group_feat in all_places[all_features]: # all places
        for single_feat in group_feat['features']:
            lon, lat = single_feat['geometry']['coordinates']
            if 'name' in single_feat['properties']:
                name = single_feat['properties']['osm_tags']['name']
            else:
                name = clnt.reverse_geocode(location=(lon, lat))['features'][0]['properties']['name']
                popup = "<strong>{0}</strong><br>Lat: {1:.3f}<br>Long: {2:.3f}".format(name, lat, lon)
            if '153' in single_feat['properties']['category_ids']:
                folium.map.Marker([lat, lon], icon=folium.Icon(color='lightgray', icon_color='#ffcc00', icon='child', prefix='fa'), popup=popup).add_to(map1)
            elif '518' in single_feat['properties']['category_ids']:
                folium.map.Marker([lat, lon], icon=folium.Icon(color='lightgray', icon_color='#009900', icon='shopping-cart', prefix='fa'), popup=popup).add_to(map1)
            else:
                folium.map.Marker([lat, lon], icon=folium.Icon(color='lightgray', icon_color='#1a1aff', icon='envelope', prefix='fa'), popup=popup).add_to(map1)

#kindergarten_group.add_to(map1)
#supermarket_group.add_to(map1)
#post_group.add_to(map1)
#map1.add_child(folium.map.LayerControl())
map1

# directions

Let's have a look at nice and short foot walking routes to each place:

In [9]:
all_house_coords = [[-122.430954, 37.792965], [-122.501636, 37.748653], [-122.446629, 37.736928]]

places_first_location = [all_kindergarten[0], all_supermarket[0], all_post[0]]
for feature_index in places_first_location:
    for geometry_index in feature_index['features']:
        coords_firstplace = geometry_index['geometry']['coordinates']
        first_direction_request = directions(client=clnt, coordinates=(all_house_coords[0], coords_firstplace), profile='foot-walking', format_out='geojson')
        folium.GeoJson(first_direction_request).add_to(map1)

places_second_location = [all_kindergarten[1], all_supermarket[1]]        
for feature_index in places_second_location:
    for geometry_index in feature_index['features']:
        coords_secondplace = geometry_index['geometry']['coordinates']
        second_direction_request = directions(client=clnt, coordinates=(all_house_coords[1], coords_secondplace), profile='foot-walking', format_out='geojson')
        folium.GeoJson(second_direction_request).add_to(map1)
        
places_third_location = [all_kindergarten[2], all_supermarket[2]]
for feature_index in places_third_location:
    for geometry_index in feature_index['features']:
        coords_thirdplace = geometry_index['geometry']['coordinates']
        third_direction_request = directions(client=clnt, coordinates=(all_house_coords[2], coords_thirdplace), profile='foot-walking', format_out='geojson')
        folium.GeoJson(third_direction_request).add_to(map1)
        
map1

## Recommended walking route

To save time we want to create a short walking route. First bring the children to the kindergarten and then do all the groceries before heading home again. 

In [32]:
route_array = [] # create list with the needed elemets for the perfect route
route_array1.append({'geometry': {'coordinates': SF1_switched}}) 
route_array1.append(kindergarten1[0])
route_array1.append(supermarket1[2]) 
route_array1.append(post1[6]) 
route_array1.append({'geometry': {'coordinates': SF1_switched}}) 

route_array.append(first_house_coords, )

In [33]:
route_direct1 = {'coordinates': [(wer['geometry']['coordinates']) for wer in route_array1],
                  'profile': 'foot-walking',
                  'preference': 'shortest',
                  'geometry': 'true',
                  'format_out': 'geojson'
                 }

all_route1 = clnt.directions(**route_direct1) # request of directions

for alle in all_route1['features']:
    all_route = alle['geometry']['coordinates']
    
all_route =[[y,x] for x,y in all_route]

folium.PolyLine(all_route, color="#c6538c").add_to(route_group) # saving it up in the route feature group

route_group.add_to(map1)

Now we have the four feature groups with all the data saved up. Using now the Layer Control we can switch on and off the different sites and routes.

For completion we will calculate the total duration for the Optimal Route. We already received the duration with the direction module and saved it up in 'all_route1'.

In [35]:
routedur1 = [(alle['properties']['segments']) for alle in all_route1['features']]
routedur2 = [(wer['duration']) for wer in routedur1[0]] 
routedur2 # duration from site to site # home - kindergarten - post office - supermarket - home

[928.2, 431.1, 10.6, 683.1]

In [36]:
sum = 0 # sum of all single durations
for num in routedur2:
    sum += num
sum # duration in seconds

2053.0

In [37]:
summe = sum / 60 # dividing by 60 to get the duration in minutes
print("Duration for Optimal Route: {:.2f} minutes.".format(summe))

Duration for Optimal Route: 34.22 minutes.
